In [1]:
import pandas as pd
import numpy as np

/Users/giorgia/.pyenv/versions/3.7.8/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
def get_daily_cases(data, col_name='series_1'):
    series = data.copy()
    series = series.groupby(by='Date_statistics').count()
    series = series.rename(columns={'Date_file': col_name})
    return series[col_name]
def make_measures_0_1(column):
    column=np.where(column==daily_data[column.name].max(),column, 0)
    column=np.where(column==0,column, 1)
    return column
    
    

In [3]:
#Load Data
measures = pd.read_excel('measures_strictness.xlsx', engine='openpyxl').rename(columns={'Unnamed: 0':'Date'})
cases_national= pd.read_csv('Data/COVID-19_casus_landelijk.csv', sep=';')
behaviour= pd.read_csv('Data/COVID-19_gedrag.csv', sep=';')

In [4]:
# Create datasets

# Put date in the right format, merge with measures data
cases_national['Date_statistics'] = pd.to_datetime(cases_national['Date_statistics'],errors='ignore', dayfirst=True)
daily_cases = get_daily_cases(cases_national, 'Number of new cases').to_frame().reset_index()
daily_cases['Date_statistics'] = pd.to_datetime(daily_cases['Date_statistics'])
daily_data = daily_cases.merge(measures, left_on='Date_statistics', right_on='Date', how='left').drop(['Date', 'stringency_index', 'government_response_index', 'containment_health_index', 'economic_support_index'], axis=1)

# Put date in the right format, filter to simplify the dataset, drop useless columns
behaviour['Date_of_measurement'] = pd.to_datetime(behaviour['Date_of_measurement'],errors='ignore')
behaviour = behaviour.loc[lambda d: (d.Region_name=='Nederland') & (d.Subgroup == 'Totaal') & (d.Sample_size != 0) & ((d.Indicator_category.str.contains('Naleving')) | (d.Indicator_category.str.contains('Zorgen')))]
behaviour= behaviour.drop(['Date_of_report', 'Wave','Region_code', 'Subgroup', 'Figure_type', 'Region_name', 'Subgroup_category', 'Lower_limit', 'Upper_limit', 'Change_wrt_previous_measurement'], axis=1)

In [5]:
new=daily_data.drop(['Date_statistics','Number of new cases'],axis=1).apply(make_measures_0_1,axis=0,result_type='expand')
daily_data=pd.concat([daily_data['Date_statistics'],daily_data['Number of new cases'],new],axis=1)
daily_data.to_csv('daily_data.csv',index=False)

In [6]:
def find_start_end_dates(measure,df):
    dates=[]
    for elem in range(len(df[measure])-1):
        if  df[measure][elem]!=df[measure][elem+1]:
            dates.append(df['Date_statistics'][elem])
    if len(dates)%2==1:
        dates.append(pd.to_datetime('2020-02-16 00:00:00'))
    return dates

measures=['c1_school_closing',
       'c2_workplace_closing', 'c3_cancel_public_events',
       'c4_restrictions_on_gatherings', 'c5_close_public_transport',
       'c6_stay_at_home_requirements', 'c7_movement_restriction',
       'c8_international_travel', 'h1_public_information_campaigns',
       'h2_testing_policy', 'h3_contact_tracing', 'h6_facial_coverings',
       'h7_vaccination_policy', 'h8_protection_of_elderly_people']
measure_dates=dict()

for measure in measures:
    measure_dates[measure]=find_start_end_dates(measure,daily_data)
    
measure_dates

{'c1_school_closing': [Timestamp('2020-03-15 00:00:00'),
  Timestamp('2020-05-10 00:00:00'),
  Timestamp('2020-12-15 00:00:00'),
  Timestamp('2021-02-07 00:00:00')],
 'c2_workplace_closing': [Timestamp('2020-11-03 00:00:00'),
  Timestamp('2020-11-21 00:00:00'),
  Timestamp('2020-12-14 00:00:00'),
  Timestamp('2020-02-16 00:00:00')],
 'c3_cancel_public_events': [Timestamp('2020-03-09 00:00:00'),
  Timestamp('2020-06-30 00:00:00'),
  Timestamp('2020-09-28 00:00:00'),
  Timestamp('2020-02-16 00:00:00')],
 'c4_restrictions_on_gatherings': [Timestamp('2020-03-22 00:00:00'),
  Timestamp('2020-05-31 00:00:00'),
  Timestamp('2020-08-17 00:00:00'),
  Timestamp('2020-02-16 00:00:00')],
 'c5_close_public_transport': [Timestamp('2020-03-22 00:00:00'),
  Timestamp('2020-05-31 00:00:00'),
  Timestamp('2020-12-14 00:00:00'),
  Timestamp('2020-02-16 00:00:00')],
 'c6_stay_at_home_requirements': [Timestamp('2020-03-22 00:00:00'),
  Timestamp('2020-05-10 00:00:00'),
  Timestamp('2021-01-22 00:00:00'),
 